In [15]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

filename = "encoded_data_03_12.csv"
df = pd.read_csv(f"./DataSets/EncodedData/{filename}", sep=",", encoding="UTF-8")
df.head()

,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
0,184.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,35,6.60206,6.445992,6.241155,6.659857,6.525779
1,184.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,35,6.30103,6.445992,6.241155,6.659857,6.525779
2,184.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,36,6.00000,6.445992,6.241155,6.659857,6.525779
3,184.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,36,6.00000,6.445992,6.241155,6.659857,6.525779
4,184.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,37,6.00000,6.445992,6.241155,6.659857,6.525779


In [16]:
df = df.sample(n=5000, random_state=42)

In [17]:
# log_market_value_base10
# Del dataene inn i treningssett og testsett
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)  # Juster test_size og random_state etter behov

# Konverter dataene til TabularDataset-objekter
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
train_data.head()



,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
62687,193.0,2015.75,5.0,2.0,0.0,0.0,0.0,1.0,180.0,24,6.845098,6.249496,6.193367,6.485577,6.525779
66460,172.0,2023.25,34.0,24.0,0.0,1.0,0.0,3.0,1665.0,31,6.954243,6.505086,6.146309,6.747129,6.431832
89172,176.0,2023.75,11.0,25.0,3.0,2.0,0.0,3.0,701.0,27,5.602060,5.702792,6.241155,5.360281,5.723697
17346,183.0,2013.25,30.0,18.0,1.0,3.0,1.0,3.0,1313.0,30,6.954243,6.299346,6.218336,6.455654,6.008928
103167,186.0,2016.25,11.0,12.0,2.0,0.0,0.0,0.0,728.0,19,6.301030,6.562910,6.241155,6.048085,5.975539


In [18]:
# Opprett en TabularPredictor og tren modellen

# 
predictor = TabularPredictor(label='log_market_value_base10', problem_type="regression", eval_metric = "mean_absolute_percentage_error").fit(train_data, presets='high_quality')

# Evaluer modellen på testsettet
performance = predictor.evaluate(test_data)
print(performance)

No path specified. Models will be saved in: "AutogluonModels\ag-20240328_145936"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240328_145936"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.11.3
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          6
Memory Avail:       5.29 GB / 15.36 GB (34.5%)
Disk Space Avail:   171.54 GB / 475.69 GB (36.1%)
Train Data Rows:    3750
Train Data Columns: 14
Label Column:       log_market_value_base10
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...


	Available Memory:                    5432.87 MB
	Train Data (Original)  Memory Usage: 0.40 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['height_in_cm', 'year', 'goals_for', 'goals_against', 'goals', ...]
		('int', [])   :  1 | ['age_at_evaluation']
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 13 | ['height_in_cm', 'year', 'goals_for', 'goals_against', 'goals', ...]
		('int', [])   :  1 | ['age_at_evaluation']
	0.1s = Fit runtime
	14 features in o

{'root_mean_squared_error': -0.28960954578010717, 'mean_squared_error': -0.08387368900696, 'mean_absolute_error': -0.2216260073268446, 'r2': 0.8005942484115138, 'pearsonr': 0.8948759948985872, 'median_absolute_error': -0.17920482196262277}


In [19]:
y_pred = predictor.predict(test_data.drop(columns=["log_market_value_base10"]))
y_pred.head()

50188     5.936003
36505     6.753777
143117    5.512286
111356    5.594072
141265    6.215724
Name: log_market_value_base10, dtype: float32

In [20]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -0.28960954578010717,
 'mean_squared_error': -0.08387368900696,
 'mean_absolute_error': -0.2216260073268446,
 'r2': 0.8005942484115138,
 'pearsonr': 0.8948759948985872,
 'median_absolute_error': -0.17920482196262277}

In [21]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT,-0.287853,-0.288712,root_mean_squared_error,0.013994,0.003000,0.878209,0.013994,0.003000,0.878209,1,True,3
1,WeightedEnsemble_L2,-0.289610,-0.285862,root_mean_squared_error,0.176625,0.069126,4.562653,0.002000,0.000000,0.187584,2,True,11
2,CatBoost,-0.291295,-0.288400,root_mean_squared_error,0.012012,0.003023,1.592119,0.012012,0.003023,1.592119,1,True,6
3,LightGBM,-0.299670,-0.291399,root_mean_squared_error,0.004999,0.001000,0.611124,0.004999,0.001000,0.611124,1,True,4
4,LightGBMLarge,-0.303724,-0.306333,root_mean_squared_error,0.023529,0.004004,1.559652,0.023529,0.004004,1.559652,1,True,10
5,XGBoost,-0.305653,-0.297536,root_mean_squared_error,0.025991,0.003014,0.622147,0.025991,0.003014,0.622147,1,True,8
6,RandomForestMSE,-0.308526,-0.296346,root_mean_squared_error,0.143620,0.062103,1.293617,0.143620,0.062103,1.293617,1,True,5
7,ExtraTreesMSE,-0.310139,-0.303435,root_mean_squared_error,0.148082,0.059886,0.675255,0.148082,0.059886,0.675255,1,True,7
8,NeuralNetTorch,-0.312557,-0.312408,root_mean_squared_error,0.015005,0.007997,11.457370,0.015005,0.007997,11.457370,1,True,9
9,KNeighborsDist,-0.576503,-0.586796,root_mean_squared_error,0.020999,0.017524,0.008991,0.020999,0.017524,0.008991,1,True,2
